In [64]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

from praenomen2genus import Genie
genie = Genie()
import pandas as pd

genie.get_gender("Eirik")
#(u'Eirik', 'man', 'list_lookup')
genie.get_gender("Ulerikka", verbose=True)
#(u'Ulerikka', 'kvinne', 'predictor')
genie.get_gender("Ellen")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
name_ Eirik len 5 <class 'str'>


AssumedGender(name='Eirik', gender='mann', mode='predictor', assumed_name='Eirik')

Ulerikka --> Ulerikka
name_ Ulerikka len 8 <class 'str'>
Finner ingenting i listene, kjør AI
resultat av 'Ulerikka':  F


AssumedGender(name='Ulerikka', gender='kvinne', mode='predictor', assumed_name='Ulerikka')

name_ Ellen len 5 <class 'str'>


AssumedGender(name='Ellen', gender='mann', mode='predictor', assumed_name='Ellen')

In [4]:
# Har navnedag
url = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager"
data = pd.read_html(url, header=0, index_col=0)[0]
#data.head()
data = data[["Navn"]]
data.dropna(subset=['Navn'], how='all', inplace = True) # skip 1.jan, er nan
data.head()

# | Navn  | dato
# | Eirik | 18 mai
har_navnedag = []
har_navnedag.append(["Jesus", "1. januar"])

rene_navn1 = []

for d in data.iterrows():
    #print(d[0], type(d[0]))
    #print(d[1].Navn)
    navn_pr_dag = [n for n in d[1].Navn.replace(",","").split() if n.lower() not in ["og", "(navn)"]]
    for n in navn_pr_dag:
        har_navnedag.append([n, d[0]])
        rene_navn1.append(n)
    #print()

df = pd.DataFrame.from_records(har_navnedag, columns=["Navn", "Dato"])
# save only this one scrape
#df.to_csv(path_or_buf="praenomen2genus/data/har_norsk_navnedag.csv", sep='\t', encoding='utf-8', index=False)

df.set_index("Navn", inplace=True)
#df.head()


#    !

# pokker finnes enda en liste:
url2 = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager_sortert_etter_navn"
data2 = pd.read_html(url2, header=0, index_col=0)[0]
data2 = data2[["Navnedag"]]
data2.columns = [["Dato"]]
#data2.head()
#data.describe()
rene_navn2 = []
for nn in data2.iterrows():
    rene_navn2.append(nn[0])

bare_i_forste = [n for n in rene_navn1 if n not in rene_navn2]
bare_i_andre = [n for n in rene_navn2 if n not in rene_navn1]

#print(len(bare_i_forste), bare_i_forste)
#print(len(bare_i_andre), bare_i_andre)
print("jeg burde ha så mange navn:", len(set(rene_navn1+rene_navn2)))


# la oss merge disse

#result = df.append(data2) #ignore_index=True # blir 1662 rader, virker for mye...
#result.describe()
#result

result2 = pd.merge(df, data2, how="outer", right_index=True, left_index=True) # 


,Navn
Dato,
2. januar,Dagfinn og Dagfrid
3. januar,Alfred og Alf og Alva
4. januar,Roar og Roger
5. januar,Hanna og Hanne
6. januar,"Aslaug, Åslaug og Aisha"


jeg burde ha så mange navn: 855


In [8]:
# clean up multiple datoer...
#result2["dato"] = result2["Dato"].fillna(result2["Dato,"]) # trick!
#result2 = result2[["dato"]] # clean date please
result2.head()



len(result2)

# lagre...
#result2.to_csv(path_or_buf="praenomen2genus/data/har_norsk_navnedag.csv", sep='\t', encoding='utf-8')#, index=False


,Dato,"(Dato,)"
Navn,,
Abraham,20. desember,20. desember
Ada,4. mars,4. mars
Adam,24. desember,24. desember
Adrian,4. mars,4. mars
Agate,5. februar,5. februar


857

In [9]:
begge_kjonn = ["Gunnleiv", "Kim"]
kvinne = ["Hallgjerd"]
menn = ["Klement", "Clement", "Clément", "Salve"]

har_navnedag_kjonn = []
for row in result2.itertuples():
    if genie.get_gender(row.Index)[2] != "list_lookup":
        if 
        print(row.Index, genie.get_gender(row.Index))
    else:
        gender = gender_string(genie.get_gender(row.Index)[2])
    
    har_navnedag_kjonn.append(gender)

har_navnedag = result2

SyntaxError: invalid syntax (<ipython-input-9-737a90f017af>, line 8)

3176

1274

509

,Fornvan,Fullt navn,kjønn
0,Abid,Abid Raja,m
1,Anja,Anja Ninasdotter Abusland,f
2,Anna,Anna Molberg,f
3,Aleksander,Aleksander Stokkebø,m
4,Alfred,Alfred Jens Bjørlo,m


['21-11-02.pickle', '21-11-09.pickle']

6352

1274

,Fornvan,Fullt navn,kjønn
1,Anja,Anja Ninasdotter Abusland,kvinne
2,Anna,Anna Molberg,kvinne
3,Aleksander,Aleksander Stokkebø,mann
4,Alfred,Alfred Jens Bjørlo,mann
5,Anne,Anne Kristine Linnestad,kvinne
...,...,...,...
3171,Ulrik,Ulrik Olsen,mann
3172,Wilhelm,Wilhelm Engel Bredal,mann
3173,Aldor,Aldor Ingebrigtsen,mann
3174,Aasmund,Aasmund Kulien,mann


In [10]:
# norrøne navn
norr_url = "https://no.wikipedia.org/wiki/Liste_over_personer_i_norr%C3%B8n_mytologi"

norr = pd.read_html(norr_url, header=0, index_col=0)[0]
norr2 = norr[["Norrønt navn"]]

norr2.head()
#norr2.to_csv(path_or_buf="praenomen2genus/data/personer_i_norron_mytologi.csv", sep='\t', encoding='utf-8')#
personer_i_norron_mytologi = norr2

,Norrønt navn
Navn,
Allvis,"Allvís, «den som vet alt»"
Andrimne,"Andrimner, «en med sot i ansiktet»"
Andvare,Andvari
Angerboda,"Angrboða, «sorgvarsleren»"
Aurboda,Eirboða


In [11]:
# Bibelske navn
url1 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-02.html" # jenter
url2 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-01.html" # gutter

bibjent = pd.read_html(url1, index_col=0)[1] #  header=0, føkket det til..
bibjent.columns = [["Kommentar"]]
bibjent.index.names = ['Navn']
bibjent.head()

bibeljenter = ['Abiah', 'Abigail', 'Abijah', 'Abilene', 'Abishag', 'Abital', 'Adah', 'Adina', 'Apphia', 'Ariel', 'Asenath', 'Ashtoreth', 'Atarah', 'Azubah', 'Basemath', 'Bashemath', 'Basmath', 'Bathsheba', 'Bernice', 'Bethany', 'Bethel', 'Beulah', 'Bilhah', 'Bithiah', 'Candace', 'Carmel', 'Chloe', 'Damaris', 'Deborah', 'Delilah', 'Dinah', 'Dorcas', 'Drusilla', 'Eden', 'Edna', 'Elisabeth', 'Elisheba', 'Elizabeth', 'Esther', 'Eunice', 'Eve', 'Gethsemane', 'Hadassah', 'Hagar', 'Hannah', 'Havilah', 'Helah', 'Hephzibah', 'Hepzibah', 'Hulda', 'Huldah', 'Ivah', 'Jael', 'Jedidah', 'Jemima', 'Jerusha', 'Jezebel', 'Joanna', 'Jordan', 'Judith', 'Julia', 'Keren-happuch', 'Keturah', 'Keziah', 'Leah', 'Lois', 'Lydia', 'Magdalene', 'Mahalath', 'Mahlah', 'Mara', 'Martha', 'Mary', 'Mehetabel', 'Mehitabel', 'Micaiah', 'Michal', 'Miriam', 'Moriah', 'Naamah', 'Naomi', 'Neriah', 'Noa', 'Noah', 'Ophrah', 'Orpah', 'Peninnah', 'Persis', 'Philadelphia', 'Phoebe', 'Prisca', 'Priscilla', 'Rachel', 'Rahab', 'Rebecca', 'Rebekah', 'Rhoda', 'Ruth', 'Salome', 'Sapphira', 'Sarah', 'Sarai', 'Sela', 'Sharon', 'Sherah', 'Shiphrah', 'Susanna', 'Susannah', 'Syntyche', 'Tabitha', 'Talitha', 'Tamar', 'Tirzah', 'Tryphena', 'Tryphosa', 'Vashti', 'Zillah', 'Zilpah', 'Zipporah']
for j in bibeljenter:
    if j not in bibjent.index:
        bibjent.loc[j] = pd.Series({'Kommentar': "fra liste"})

bibjent["Kjønn"] = "f"
bibjent.tail()


bibgutt = pd.read_html(url2, header=0, index_col=0)[1]
bibgutt.columns = [["Kommentar"]]
bibgutt.index.names = ['Navn']
bibgutt["Kjønn"] = "m"
bibgutt.head()

IndexError: list index out of range

In [171]:
# Jødiske navn

In [173]:
# Populære navn (periodisk oppdatet fra ssb?)
# for mange land her: http://www.uib.no/lle/23822/navnestatistikk-fra-mange-land

In [ ]:
# Som rimer på
# som likner på
# som inneholder en "o"
# som besteforeldrenes generasjon brukte
# som olderforeldrenes generasjon brukte
# som tip-tip-tip...


In [213]:
from pandas.io import sql
import pymysql
def connect():
    try:
        rdbms_username = 'root'#settings['user']
        rdbms_password = 'root'#settings['password']
        connection = pymysql.connect(
            unix_socket="/Applications/MAMP/tmp/mysql/mysql.sock",
            user=rdbms_username,
            passwd=rdbms_password,
            db="navn", charset='utf8',
            autocommit=True)
        cur = connection.cursor()
        cur.execute("USE navn;")
        return connection, cur
    except:
        print("kunne ikke logge på databasen")


def disconnect(connection):
    if connection:
        connection.close()
        print("koblet av mysql")

mysql_connection, mysql_cur = connect()
#sql = '''SELECT CONCAT(title, " ", full_text) as text, url, man_clas FROM
#nrk2013b_man_vs_mac_tbl WHERE man_clas IS NOT NULL'''
#mysql_cur.execute(sql)
#rows = mysql_cur.fetchall()


In [284]:
# lagre i db

def df2mysql(df, sql_table_name, gender_row="Kjønn"):
    mysql_connection, mysql_cur = connect()
    #df.to_sql(sql_table_name, mysql_connection, flavor=None, schema=None, if_exists='fail', index=True, index_label=index_label, chunksize=None, dtype=None)
    for row in df.iterrows():
        #print(row[0], row[1]["Kjønn"])
        #print()
        sql = "REPLACE INTO `{0}` (`name`, `gender`) VALUES ('{1}', '{2}');".format(sql_table_name, row[0], row[1][gender_row])        #print(sql)

        mysql_cur.execute(sql);
    #mysql_cur.commit()

#bibjent.head()
#bibgutt.head()
#df2mysql(bibjent, sql_table_name="bibelske", gender_row="Kjønn")    
#df2mysql(bibgutt, sql_table_name="bibelske", gender_row="Kjønn")    

har_navnedag.head()
#df2mysql(bibgutt, sql_table_name="bibelske", gender_row="Kjønn")    


stortingsrepresentantnavn.head()
personer_i_norron_mytologi.head()


,dato
Navn,
Abraham,20. desember
Ada,4. mars
Adam,24. desember
Adrian,4. mars
Agate,5. februar


,Fornvan,Fullt navn,kjønn
0,Abid,Abid Q. Raja,m
1,Anna,Anna Ljunggren,f
2,Audun,Audun Lysbakken,m
3,Anders,Anders Anundsen,m
4,Anniken,Anniken Huitfeldt,f


,Norrønt navn
Navn,
Allvis,"Allvís, «den som vet alt»"
Andrimne,"Andrimner, «en med sot i ansiktet»"
Andvare,Andvari
Angerboda,"Angrboða, «sorgvarsleren»"
Aurboda,Eirboða
